# Profitable App Profiles for the Apple Store and Google Play Markets

In this project, the aim is to find mobile application profiles that are profitable for the Apple Store and Google Play markets. 


## Opening and Exploring the Data

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. These are two data sets that seem suitable for our purpose:

- [A data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
- [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store

Let's start by opening the two data sets and then continue with exploring them.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

android = pd.read_csv('googleplaystore.csv')
ios = pd.read_csv('applestore.csv')

In [2]:
android.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are `'App'`, `'Category'`, `'Reviews'`, `'Installs'`, `'Type'`, `'Price'`, and `'Genres'`.

Now let's take a look at the App Store data set.

In [3]:
ios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7197 entries, 0 to 7196
Data columns (total 16 columns):
id                  7197 non-null int64
track_name          7197 non-null object
size_bytes          7197 non-null int64
currency            7197 non-null object
price               7197 non-null float64
rating_count_tot    7197 non-null int64
rating_count_ver    7197 non-null int64
user_rating         7197 non-null float64
user_rating_ver     7197 non-null float64
ver                 7197 non-null object
cont_rating         7197 non-null object
prime_genre         7197 non-null object
sup_devices.num     7197 non-null int64
ipadSc_urls.num     7197 non-null int64
lang.num            7197 non-null int64
vpp_lic             7197 non-null int64
dtypes: float64(3), int64(8), object(5)
memory usage: 899.7+ KB


We have 7197 iOS apps in this data set, and the columns that seem interesting are: `'track_name'`, `'currency'`, `'price'`, `'rating_count_tot'`, `'rating_count_ver'`, and `'prime_genre'`. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).


## Deleting Wrong Data

The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [4]:
print(android.iloc[10472])   # incorrect row

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                                 19
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object


In [5]:
print(android.iloc[0])  # correct row

App               Photo Editor & Candy Camera & Grid & ScrapBook
Category                                          ART_AND_DESIGN
Rating                                                       4.1
Reviews                                                      159
Size                                                         19M
Installs                                                 10,000+
Type                                                        Free
Price                                                          0
Content Rating                                          Everyone
Genres                                              Art & Design
Last Updated                                     January 7, 2018
Current Ver                                                1.0.0
Android Ver                                         4.0.3 and up
Name: 0, dtype: object


The row 10472 corresponds to the app _Life Made WI-Fi Touchscreen Photo Frame_, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. As a consequence, we'll delete this row.

In [6]:
print(len(android))
android.drop([10472], axis = 0, inplace = True) # don't run this more than once
print(len(android))

10841
10840


## Removing Duplicate Entries

### Part One
If we explore the Google Play data set long enough, we'll find that some apps have more than one entry. For instance, the application Instagram has four entries:

In [7]:
android[android['App'] == 'Instagram']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2545,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
2604,Instagram,SOCIAL,4.5,66577446,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
2611,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
3909,Instagram,SOCIAL,4.5,66509917,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device


In total, there are 1,181 cases where an app occurs more than once:

In [8]:
duplicate_apps = []
unique_apps = []

for row in range(android.shape[0]):
    name = android.iloc[row, 0]
    if name in unique_apps:
        duplicate_apps.append(name)
    unique_apps.append(name)
    
print(len(duplicate_apps))
print(duplicate_apps[:15])

1181
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed two cells above for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

To do that, we will:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

### Part Two

Let's start by building the dictionary.

In [9]:
reviews_max = {}

for row in range(android.shape[0]):
    name = android.iloc[row, 0]
    n_reviews = float(android.iloc[row, 3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In a previous code cell, we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and 1,181.

In [10]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Now, let's use the `reviews_max` dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

- We start by initializing two empty lists, `android_clean` and `already_added`.
- We loop through the `android` data set, and for every iteration:
    - We isolate the name of the app and the number of reviews.
    - We add the current row (`app`) to the `android_clean` list, and the app name (`name`) to the `already_cleaned` list if:
        - The number of reviews of the current app matches the number of reviews of that app as described in the `reviews_max` dictionary; and
        - The name of the app is not already in the `already_added` list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name] == n_reviews`, we'll still end up with duplicate entries for some apps.

In [11]:
android_clean = pd.DataFrame()
already_added = []

for row in range(android.shape[0]):
    name = android.iloc[row, 0]
    n_reviews = float(android.iloc[row, 3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean = android_clean.append(android.iloc[row], ignore_index=True)
        already_added.append(name) # make sure this is inside the if block

Now let's quickly explore the dataframe `android_clean`, and confirm that the number of rows is 9,659.

In [12]:
android_clean.head()

,Android Ver,App,Category,Content Rating,Current Ver,Genres,Installs,Last Updated,Price,Rating,Reviews,Size,Type
0,4.0.3 and up,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,Everyone,1.0.0,Art & Design,"10,000+","January 7, 2018",0,4.1,159,19M,Free
1,4.0.3 and up,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,Everyone,1.2.4,Art & Design,"5,000,000+","August 1, 2018",0,4.7,87510,8.7M,Free
2,4.2 and up,Sketch - Draw & Paint,ART_AND_DESIGN,Teen,Varies with device,Art & Design,"50,000,000+","June 8, 2018",0,4.5,215644,25M,Free
3,4.4 and up,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,Everyone,1.1,Art & Design;Creativity,"100,000+","June 20, 2018",0,4.3,967,2.8M,Free
4,2.3 and up,Paper flowers instructions,ART_AND_DESIGN,Everyone,1.0,Art & Design,"50,000+","March 26, 2017",0,4.4,167,5.6M,Free


In [13]:
print(len(android_clean))

9659


We have 9659 rows, just as expected.

## Removing Non-English Apps

### Part One

If you explore the data sets enough, you'll notice the names of some of the applications suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [14]:
print(ios.iloc[813,1])
print(ios.iloc[6731,1])

print(android_clean.iloc[4412,1])
print(android_clean.iloc[7940,1])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of applications, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an application name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in `ord()` function to find out the corresponding encoding number of each character.

In [15]:
def is_English(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_English('Instagram'))
print(is_English('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

In [16]:
print(is_English('Docs To Go™ Free Office Suite'))
print(is_English('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


### Part Two

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [17]:
def is_English(row):
    non_ascii = 0
    string = row[1]
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the `is_English()` function to filter out the non-English apps for both data sets:

In [18]:
android_english = android_clean[android_clean.apply(is_English, axis=1)]
ios_english = ios[ios.apply(is_English, axis=1)]

In [19]:
android_english.head()

,Android Ver,App,Category,Content Rating,Current Ver,Genres,Installs,Last Updated,Price,Rating,Reviews,Size,Type
0,4.0.3 and up,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,Everyone,1.0.0,Art & Design,"10,000+","January 7, 2018",0,4.1,159,19M,Free
1,4.0.3 and up,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,Everyone,1.2.4,Art & Design,"5,000,000+","August 1, 2018",0,4.7,87510,8.7M,Free
2,4.2 and up,Sketch - Draw & Paint,ART_AND_DESIGN,Teen,Varies with device,Art & Design,"50,000,000+","June 8, 2018",0,4.5,215644,25M,Free
3,4.4 and up,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,Everyone,1.1,Art & Design;Creativity,"100,000+","June 20, 2018",0,4.3,967,2.8M,Free
4,2.3 and up,Paper flowers instructions,ART_AND_DESIGN,Everyone,1.0,Art & Design,"50,000+","March 26, 2017",0,4.4,167,5.6M,Free


In [20]:
print(len(android_english))

9614


In [21]:
ios_english.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1


In [22]:
print(len(ios_english))

6183


We can see that we're left with 9614 Android apps and 6183 iOS apps.

## Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [23]:
android_english['Price'] = android_english['Price'].str.replace('$','').astype(float)
android_final = android_english[android_english['Price'] == 0]
ios_final = ios_english[ios_english['price'] == 0]
       
print(len(android_final))
print(len(ios_final))

8864
3222


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## Most Common Apps by Genre

### Part One

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

### Part Two

We start by examining the frequency table for the `prime_genre` column of the App Store data set.

In [24]:
round(ios_final['prime_genre'].value_counts(normalize = True)*100, 2)

Games                58.16
Entertainment         7.88
Photo & Video         4.97
Education             3.66
Social Networking     3.29
Shopping              2.61
Utilities             2.51
Sports                2.14
Music                 2.05
Health & Fitness      2.02
Productivity          1.74
Lifestyle             1.58
News                  1.33
Travel                1.24
Finance               1.12
Weather               0.87
Food & Drink          0.81
Reference             0.56
Business              0.53
Book                  0.43
Navigation            0.19
Medical               0.19
Catalogs              0.12
Name: prime_genre, dtype: float64

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set. 

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer. 

Let's continue by examining the `Genres` and `Category` columns of the Google Play data set (two columns which seem to be related).

In [25]:
round(android_final['Category'].value_counts(normalize = True)*100, 2)

FAMILY                 18.91
GAME                    9.72
TOOLS                   8.46
BUSINESS                4.59
LIFESTYLE               3.90
PRODUCTIVITY            3.89
FINANCE                 3.70
MEDICAL                 3.53
SPORTS                  3.40
PERSONALIZATION         3.32
COMMUNICATION           3.24
HEALTH_AND_FITNESS      3.08
PHOTOGRAPHY             2.94
NEWS_AND_MAGAZINES      2.80
SOCIAL                  2.66
TRAVEL_AND_LOCAL        2.34
SHOPPING                2.25
BOOKS_AND_REFERENCE     2.14
DATING                  1.86
VIDEO_PLAYERS           1.79
MAPS_AND_NAVIGATION     1.40
FOOD_AND_DRINK          1.24
EDUCATION               1.16
ENTERTAINMENT           0.96
LIBRARIES_AND_DEMO      0.94
AUTO_AND_VEHICLES       0.93
HOUSE_AND_HOME          0.82
WEATHER                 0.80
EVENTS                  0.71
PARENTING               0.65
ART_AND_DESIGN          0.64
COMICS                  0.62
BEAUTY                  0.60
Name: Category, dtype: float64

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

<center>
![img](https://s3.amazonaws.com/dq-content/350/py1m8_family.png)
Source: [play.google.com](https://play.google.com/store/apps/category/FAMILY?hl=en)
</center>

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the `Genres` column:

In [26]:
round(android_final['Genres'].value_counts(normalize = True)*100, 2)

Tools                                    8.45
Entertainment                            6.07
Education                                5.35
Business                                 4.59
Lifestyle                                3.89
Productivity                             3.89
Finance                                  3.70
Medical                                  3.53
Sports                                   3.46
Personalization                          3.32
Communication                            3.24
Action                                   3.10
Health & Fitness                         3.08
Photography                              2.94
News & Magazines                         2.80
Social                                   2.66
Travel & Local                           2.32
Shopping                                 2.25
Books & Reference                        2.14
Simulation                               2.04
Dating                                   1.86
Arcade                            

The difference between the `Genres` and the `Category` columns is not crystal clear, but one thing we can notice is that the `Genres` column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the `Category` column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [27]:
ios_final.groupby('prime_genre')['rating_count_tot'].agg(np.mean).sort_values(ascending = False)

prime_genre
Navigation           86090.333333
Reference            74942.111111
Social Networking    71548.349057
Music                57326.530303
Weather              52279.892857
Book                 39758.500000
Food & Drink         33333.923077
Finance              31467.944444
Photo & Video        28441.543750
Travel               28243.800000
Shopping             26919.690476
Health & Fitness     23298.015385
Sports               23008.898551
Games                22788.669691
News                 21248.023256
Productivity         21028.410714
Utilities            18684.456790
Lifestyle            16485.764706
Entertainment        14029.830709
Business              7491.117647
Education             7003.983051
Catalogs              4004.000000
Medical                612.000000
Name: rating_count_tot, dtype: float64

On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [28]:
# print name and number of ratings
ios_final[ios_final['prime_genre'] == 'Navigation'][['track_name', 'rating_count_tot']]

,track_name,rating_count_tot
49,"Waze - GPS Navigation, Maps & Real-time Traffic",345046
130,Google Maps - Navigation & Transit,154911
881,Geocaching®,12811
1633,CoPilot GPS – Car Navigation & Offline Maps,3582
3987,ImmobilienScout24: Real Estate Search in Germany,187
6033,Railway Route Search,5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [29]:
ios_final[ios_final['prime_genre'] == 'Reference'][['track_name', 'rating_count_tot']]

,track_name,rating_count_tot
6,Bible,985920
90,Dictionary.com Dictionary & Thesaurus,200047
335,Dictionary.com Dictionary & Thesaurus for iPad,54175
551,Google Translate,26786
715,"Muslim Pro: Ramadan 2017 Prayer Times, Azan, Q...",18418
738,New Furniture Mods - Pocket Wiki & Game Tools ...,17588
757,Merriam-Webster Dictionary,16849
913,Night Sky,12122
1106,City Maps for Minecraft PE - The Best Maps for...,8535
1451,LUCKY BLOCK MOD ™ for Minecraft PC Edition - T...,4693


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [30]:
round(android_final['Installs'].value_counts(normalize = True)*100, 2) # the Installs columns

1,000,000+        15.73
100,000+          11.55
10,000,000+       10.55
10,000+           10.20
1,000+             8.39
100+               6.92
5,000,000+         6.83
500,000+           5.56
50,000+            4.77
5,000+             4.51
10+                3.54
500+               3.25
50,000,000+        2.30
100,000,000+       2.13
50+                1.92
5+                 0.79
1+                 0.51
500,000,000+       0.27
1,000,000,000+     0.23
0+                 0.05
0                  0.01
Name: Installs, dtype: float64

One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to `float` — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [31]:
categories_android = android_final['Category'].unique()
for category in categories_android:
    total = 0
    len_category = 0
    for row in range(android_final.shape[0]):
        category_row = android_final.iloc[row, 2]
        if category_row == category:            
            n_installs = android_final.iloc[row, 6]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [32]:
android_final[(android_final['Category'] == 'COMMUNICATION') & 
              android_final['Installs'].isin(['1,000,000,000+', '500,000,000+','100,000,000+'])][['App','Installs']]

,App,Installs
279,WhatsApp Messenger,"1,000,000,000+"
282,imo beta free calls and text,"100,000,000+"
295,Android Messages,"100,000,000+"
296,Google Duo - High Quality Video Calls,"500,000,000+"
304,Messenger – Text and Video Chat for Free,"1,000,000,000+"
305,imo free video calls and chat,"500,000,000+"
308,Skype - free IM & video calls,"1,000,000,000+"
309,Who,"100,000,000+"
310,"GO SMS Pro - Messenger, Free Themes, Emoji","100,000,000+"
316,LINE: Free Calls & Messages,"500,000,000+"


In [33]:
android_final[android_final['Category'] == 'BOOKS_AND_REFERENCE'][['App','Installs']]

,App,Installs
135,E-Book Read - Read Book for free,"50,000+"
136,Download free book with green book,"100,000+"
137,Wikipedia,"10,000,000+"
138,Cool Reader,"10,000,000+"
139,Free Panda Radio Music,"100,000+"
140,Book store,"1,000,000+"
141,FBReader: Favorite Book Reader,"10,000,000+"
142,English Grammar Complete Handbook,"500,000+"
143,Free Books - Spirit Fanfiction and Stories,"1,000,000+"
144,Google Play Books,"1,000,000,000+"


The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [34]:
android_final[(android_final['Category'] == 'BOOKS_AND_REFERENCE') & 
              android_final['Installs'].isin(['1,000,000,000+', '500,000,000+','100,000,000+'])][['App','Installs']]

,App,Installs
144,Google Play Books,"1,000,000,000+"
2977,Bible,"100,000,000+"
3086,Amazon Kindle,"100,000,000+"
3650,Wattpad 📖 Free Books,"100,000,000+"
4539,Audiobooks from Audible,"100,000,000+"


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [35]:
android_final[(android_final['Category'] == 'BOOKS_AND_REFERENCE') & 
              android_final['Installs'].isin(['1,000,000+', '5,000,000+','10,000,000+','50,000,000'])][['App','Installs']] 

,App,Installs
137,Wikipedia,"10,000,000+"
138,Cool Reader,"10,000,000+"
140,Book store,"1,000,000+"
141,FBReader: Favorite Book Reader,"10,000,000+"
143,Free Books - Spirit Fanfiction and Stories,"1,000,000+"
145,AlReader -any text book reader,"5,000,000+"
148,FamilySearch Tree,"1,000,000+"
149,Cloud of Books,"1,000,000+"
151,ReadEra – free ebook reader,"1,000,000+"
153,Ebook Reader,"5,000,000+"


This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.


## Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.